In [1]:
import open3d as o3d
import numpy as np
import math
import copy

In [2]:
from pyquaternion import Quaternion


def matrix2rotation(matrix):
    return matrix[:3, :3]


def matrix2qua(matrix):
    qua = Quaternion(matrix=matrix2rotation(matrix))
    return qua


def matrix2trans(matrix):
    return np.array(matrix[:3, 3:].transpose())[0]


def matrix2pose(matrix):
    qua = matrix2qua(matrix)
    translation = matrix2trans(matrix)
    return np.hstack((translation, qua.elements))


def quatrans2matrix(qua, trans):
    rotaion_matrix = qua.rotation_matrix
    trans_matrix = np.array([trans]).transpose()
    tf_m = np.hstack((rotaion_matrix, trans_matrix))
    m = np.mat([[0, 0, 0, 1]])
    tf_m = np.vstack((tf_m, m))

    return tf_m


def pose2matrix(pose):
    w, x, y, z = pose[3:]
    qua = Quaternion(w=w, x=x, y=y, z=z)
    trans = pose[:3]
    return quatrans2matrix(qua, trans)


def translation2matrix(trans):
    qua = Quaternion(w=1, x=0, y=0, z=0)
    return quatrans2matrix(qua, trans)


def qua2matrix(qua):
    trans = [0, 0, 0]
    return quatrans2matrix(qua, trans)

In [3]:
def sample_mesh_to_cloud(mesh, sample_number=100000, color = [1, 0.706, 0]):
    """
    Description
    -----------
        process the cabinet mesh model, sample it as point cloud

    Parameters
    ----------
        mesh: o3d.geometry.TriangleMesh
        sample_number: int

    Returns
    -------
        o3d.geometry.PointCloud
    """
    mesh.compute_vertex_normals()
    pcd = mesh.sample_points_uniformly(number_of_points=sample_number)
    pcd.paint_uniform_color(color)
    return pcd

In [4]:
cloud = o3d.geometry.PointCloud()

In [5]:
box = o3d.geometry.TriangleMesh.create_box(width=0.47, height=0.36, depth=0.26)
box_cloud = sample_mesh_to_cloud(box, 5000)
box_cloud.transform(pose2matrix([2.908, -0.740, 1.378, 1, 0, 0, 0]).A)
cloud += box_cloud

box1 = o3d.geometry.TriangleMesh.create_box(width=0.47, height=0.36, depth=0.26)
box_cloud1 = sample_mesh_to_cloud(box1, 5000, [0.5,0.5,0.5])
box_cloud1.transform(pose2matrix([2.905, -0.732, 1.119, 1, 0, 0, 0]).A)
cloud += box_cloud1

box = o3d.geometry.TriangleMesh.create_box(width=0.47, height=0.36, depth=0.26)
box_cloud = sample_mesh_to_cloud(box, 5000)
box_cloud.transform(pose2matrix([2.904, -0.361, 0.872, 1, 0, 0, 0]).A)
cloud += box_cloud

box = o3d.geometry.TriangleMesh.create_box(width=0.47, height=0.36, depth=0.26)
box_cloud = sample_mesh_to_cloud(box, 5000)
box_cloud.transform(pose2matrix([2.904, -0.737, 0.869, 1, 0, 0, 0]).A)
cloud += box_cloud

box = o3d.geometry.TriangleMesh.create_box(width=0.47, height=0.36, depth=0.26)
box_cloud = sample_mesh_to_cloud(box, 5000)
box_cloud.transform(pose2matrix([2.913, -0.360, 0.615, 1, 0, 0, 0]).A)
cloud += box_cloud

box = o3d.geometry.TriangleMesh.create_box(width=0.47, height=0.36, depth=0.26)
box_cloud = sample_mesh_to_cloud(box, 5000)
box_cloud.transform(pose2matrix([2.913, -0.739, 0.606, 1, 0, 0, 0]).A)
cloud += box_cloud

box = o3d.geometry.TriangleMesh.create_box(width=0.47, height=0.36, depth=0.26)
box_cloud = sample_mesh_to_cloud(box, 5000)
box_cloud.transform(pose2matrix([2.907, -1.109, 0.606, 1, 0, 0, 0]).A)
cloud += box_cloud

box = o3d.geometry.TriangleMesh.create_box(width=0.47, height=0.36, depth=0.26)
box_cloud = sample_mesh_to_cloud(box, 5000)
box_cloud.transform(pose2matrix([2.915, -0.360, 0.358, 1, 0, 0, 0]).A)
cloud += box_cloud

box = o3d.geometry.TriangleMesh.create_box(width=0.47, height=0.36, depth=0.26)
box_cloud = sample_mesh_to_cloud(box, 5000)
box_cloud.transform(pose2matrix([2.914, -0.739, 0.355, 1, 0, 0, 0]).A)
cloud += box_cloud

box = o3d.geometry.TriangleMesh.create_box(width=0.47, height=0.36, depth=0.26)
box_cloud = sample_mesh_to_cloud(box, 5000)
box_cloud.transform(pose2matrix([2.907, -1.109, 0.356, 1, 0, 0, 0]).A)
cloud += box_cloud

In [18]:
cloud = o3d.geometry.PointCloud()

# sign_on_front = o3d.geometry.TriangleMesh.create_box(width=0.01, height=0.01, depth=0.01)
# sign_cloud = sample_mesh_to_cloud(sign_on_front, 1000)
# sign_cloud.transform(pose2matrix([0, 0, 0, 1, 0, 0, 0]).A)
# cloud += sign_cloud

world_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(1)

# sign_on_front = o3d.geometry.TriangleMesh.create_box(width=0.01, height=0.01, depth=0.01)
# sign_cloud = sample_mesh_to_cloud(sign_on_front, 1000)
# sign_cloud.transform(pose2matrix([1, 0, 0, 1, 0, 0, 0]).A)
# cloud += sign_cloud

# sign_on_front = o3d.geometry.TriangleMesh.create_box(width=0.01, height=0.01, depth=0.01)
# sign_cloud = sample_mesh_to_cloud(sign_on_front, 1000)
# sign_cloud.transform(pose2matrix([0, 1, 0, 1, 0, 0, 0]).A)
# cloud += sign_cloud

# sign_on_front = o3d.geometry.TriangleMesh.create_box(width=0.01, height=0.01, depth=0.01)
# sign_cloud = sample_mesh_to_cloud(sign_on_front, 1000)
# sign_cloud.transform(pose2matrix([0, 0, 1, 1, 0, 0, 0]).A)
# cloud += sign_cloud

box = o3d.geometry.TriangleMesh.create_box(width=0.135, height=0.27, depth=0.4)
box_cloud = sample_mesh_to_cloud(box, 10000)
box_cloud.transform(pose2matrix([0.432306, -1.79341, -0.4 + 0.39580, 1,0,0,0]).A)
cloud += box_cloud

box = o3d.geometry.TriangleMesh.create_box(width=0.135, height=0.27, depth=0.4)
box_cloud = sample_mesh_to_cloud(box, 10000)
box_cloud.transform(pose2matrix([0.432306-0.137, -1.79341, -0.4 + 0.39580, 1,0,0,0]).A)
cloud += box_cloud

box = o3d.geometry.TriangleMesh.create_box(width=0.39, height=0.425, depth=0.6)
box_cloud = sample_mesh_to_cloud(box, 10000)
box_cloud.transform(pose2matrix([0.26643 -0.39, -1.52622 -0.425, -0.6 + 0.675786, 1,0,0,0]).A)
cloud += box_cloud

box = o3d.geometry.TriangleMesh.create_box(width=0.27, height=0.135, depth=0.19)
box_cloud = sample_mesh_to_cloud(box, 10000)
box_cloud.transform(pose2matrix([0.22574-0.27, -1.60253-0.135, -0.19 + 0.864713, 1,0,0,0]).A)
cloud += box_cloud

box = o3d.geometry.TriangleMesh.create_box(width=0.405, height=0.62, depth=0.7)
box_cloud = sample_mesh_to_cloud(box, 10000)
box_cloud.transform(pose2matrix([-0.5299, -1.39576-0.62, -0.7 + 1.12699, 1,0,0,0]).A)
cloud += box_cloud

box = o3d.geometry.TriangleMesh.create_box(width=0.27, height=0.19, depth=0.135)
box_cloud = sample_mesh_to_cloud(box, 10000)
box_cloud.transform(pose2matrix([-0.50507, -1.49368-0.19, -0.135 + 1.26338, 1,0,0,0]).A)
cloud += box_cloud

o3d.visualization.draw_geometries([world_frame, cloud])


In [35]:
world_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(1)

cloud = o3d.geometry.PointCloud()

box = o3d.geometry.TriangleMesh.create_box(width=1, height=1.2, depth=0.01)
box_cloud = sample_mesh_to_cloud(box, 100000)
box_cloud.transform(pose2matrix([-0.5, -0.6, 0.01, 1,0,0,0]).A)
box_cloud.transform(pose2matrix([-0.57428, 0.0153397, 1.80461, 0.999183, 0.036001, -0.00715141, 0.0169332]).A)
cloud += box_cloud

add_sign = True
if add_sign:
    sign_on_front = o3d.geometry.TriangleMesh.create_box(width=0.01, height=0.01, depth=0.01)
    sign_cloud = sample_mesh_to_cloud(sign_on_front, 1000)
    sign_cloud.transform(pose2matrix([-2, 0, 1.5, 1, 0, 0, 0]).A)
    cloud =  cloud + sign_cloud
# o3d.visualization.draw_geometries([target])

o3d.visualization.draw_geometries([world_frame, cloud, source])

In [20]:
o3d.visualization.draw_geometries([cloud])

In [36]:
target = cloud

In [24]:

add_sign = True
if add_sign:
    sign_on_front = o3d.geometry.TriangleMesh.create_box(width=0.01, height=0.01, depth=0.01)
    sign_cloud = sample_mesh_to_cloud(sign_on_front, 1000)
#     sign_cloud.transform(pose2matrix([1.75 + 1.2, 0, 0.07 + 0.26, 1, 0, 0, 0]).A)
    sign_cloud.transform(pose2matrix([3.5, 0, 1, 1, 0, 0, 0]).A)
    target =  target + sign_cloud
o3d.visualization.draw_geometries([target])

In [5]:
def load_point_cloud(path):
    """
    Description
    -----------
        read point cloud

    Parameters
    ----------
        path: str

    Returns
    -------
        o3d.geometry.PointCloud
    """
    cloud = o3d.io.read_point_cloud(path)
    return cloud

def get_picked_indices(pcd):
    vis = o3d.visualization.VisualizerWithEditing()
    vis.create_window(window_name="Resgistration: Pick Corresponding Points")
    vis.add_geometry(pcd)
    vis.run()
    vis.destroy_window()
    return vis.get_picked_points()


def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries(
        [source_temp, target], window_name="Registration Visualization"
    )
    return source_temp + target

In [37]:
source = load_point_cloud("/home/bot/dev/projects_data/honghe/pallet_on_site/tfed.pcd")
source_ids = get_picked_indices(source)
# source_ids = [18846, 19812, 40421, 40864, 41687, 42122, 42167, 43343, 69414, 69115, 68812]
print(len(source_ids))
print(source_ids)

4
[8020, 318401, 327486, 9248]


In [38]:
target_ids = get_picked_indices(target)
# target_ids = [31252, 46024, 1273, 1674, 18570, 25855, 26750, 40816, 11751, 38898, 36067]
print(len(target_ids))
print(target_ids)

4
[366, 1333, 1191, 525]


In [39]:
corr = np.zeros((len(source_ids), 2))
corr[:, 0] = source_ids
corr[:, 1] = target_ids

p2p = o3d.registration.TransformationEstimationPointToPoint()
trans_init = p2p.compute_transformation(
    source, target, o3d.utility.Vector2iVector(corr)
)
print(trans_init)
draw_registration_result(source, target, trans_init)

[[ 9.99815791e-01  3.27710237e-03 -1.89114806e-02  3.28539000e-02]
 [-3.27468453e-03  9.99994626e-01  1.58816114e-04 -7.43439048e-03]
 [ 1.89118994e-02 -9.68577253e-05  9.99821149e-01  4.56040796e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


geometry::PointCloud with 428651 points.

In [40]:
reg_p2p = o3d.registration.registration_icp(
    source,
    target,
    0.005,
    trans_init,
    o3d.registration.TransformationEstimationPointToPoint(),
)

transform = reg_p2p.transformation
print("Transformation Matrix: \n{}\n".format(transform))

draw_registration_result(source, target, transform)

pose = matrix2pose(transform)
print(pose)

Transformation Matrix: 
[[ 9.99970996e-01  2.28276007e-03 -7.26605749e-03  1.21522882e-02]
 [-2.27848260e-03  9.99997226e-01  5.96915763e-04 -7.73266808e-03]
 [ 7.26739995e-03 -5.80342865e-04  9.99973424e-01  4.09489393e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

[ 1.21522882e-02 -7.73266808e-03  4.09489393e-02  9.99992706e-01
 -2.94316804e-04 -3.63339087e-03 -1.14031899e-03]


In [ ]:
pose = [ 0.8227459,  -0.86084309,  2.04105511,  0.60288354, -0.0439842 ,  0.79606347,
  0.02966113]
matrix = pose2matrix(pose)
print(matrix)

draw_registration_result(source, target, matrix)